# Get the hex centroids by clicking the center of each hex on the video

We click each hex individually instead of fitting a hex grid because the camera is slightly angled such that the hex grid doesn't fit perfectly onto the maze video. This gives us better results (fitting the hex grid ends up warping space).

### Important: hexes MUST be clicked from top top bottom, left to right!

If hex 1 is on top in the maze, this looks like: hex 1, 4, 6, 5, 8, 7, 11, etc. 

Set the correct `hex_list` based on which hex is on the top in your video (1, 2, or 3), and set `video_file_path` to point to your video file.

Click only the 49 potentially open hexes (not locations where permanent barriers will be placed).

To undo a click, press `u` for `undo`.

When all hexes have been clicked, press `q` or `escape` to close the video window.

Then scroll to the next code block to save the clicked hex centroids as a csv file.

In [1]:
import cv2
import pandas as pd
import numpy as np

# Hexes from top to bottom, left to right, if hex 1 is on top
HEX_LIST_1 = [1, 4, 6, 5, 8, 7, 11, 10, 9, 14, 13, 12, 18, 17, 16, 15,
            22, 21, 20, 19, 27, 26, 25, 24, 23, 32, 31, 30, 29, 28,
            38, 37, 36, 35, 34, 33, 49, 42, 41, 40, 39, 48, 2, 47, 46, 45, 44, 43, 3]
# Hexes from top to bottom, left to right, if hex 2 is on top
HEX_LIST_2 = [2, 49, 47, 38, 42, 32, 46, 37, 27, 41, 31, 22, 45, 36, 26, 18, 
              40, 30, 21, 14, 44, 35, 25, 17, 11, 39, 29, 20, 13, 8, 
              43, 34, 24, 16, 10, 6, 48, 28, 19, 12, 7, 4, 3, 33, 23, 15, 9, 5, 1]
# Hexes from top to bottom, left to right, if hex 3 is on top
HEX_LIST_3 = [3, 48, 33, 43, 28, 39, 23, 34, 44, 19, 29, 40, 15, 24, 35, 45, 
              12, 20, 30, 41, 9, 16, 25, 36, 46, 7, 13, 21, 31, 42, 
              5, 10, 17, 26, 37, 47, 4, 8, 14, 22, 32, 49, 1, 6, 11, 18, 27, 38, 2]

# Berke Lab video angle shows the maze with hex 1 on top, so use HEX_LIST_1
# Frank Lab video angle shows the maze with hex 2 on top, so use HEX_LIST_2
hex_list = HEX_LIST_1

In [ ]:
# Path to the hex maze video to get centroids from
video_file_path = 'frank_lab/video_files/maze_empty.h264'

# CM_PER_PIXEL is optional.
# For Berke Lab, set CM_PER_PIXEL = None 
# (we do pixels_per_cm assignment elsewhere, so if you set a value here it won't end up in the nwbfile anyway)
# For Frank Lab, CM_PER_PIXEL = 0.16 for Xulu's latest maze. But setting CM_PER_PIXEL = None is also fine,
# because we do hex assignment using raw pixel position.
# Keeping for posterity for now - I will probably remove this entirely in the future so this code is cleaner.
CM_PER_PIXEL = None

# Declare frame and clicked_points as global variables
frame = None
clicked_points = []

def get_pixel_coordinates(event, x, y, flags, param):
    """ Callback function to get the (x, y) coordinates of a click """
    if event == cv2.EVENT_LBUTTONDOWN: # Left mouse button click
        hex_index = len(clicked_points)
        if hex_index >= len(hex_list):
            print("All hexes have already been clicked!!")
            print("Press 'q' or 'escape' to finish, or 'u' to undo clicks.")
            return
        # Print and store the clicked point
        clicked_points.append((x, y))
        hex_number = hex_list[hex_index]
        print(f"Clicked hex {hex_number} at ({x}, {y})")
        # Draw a red dot at the clicked position so we know where we clicked
        cv2.circle(frame, (x, y), radius=5, color=(0, 0, 255), thickness=-1)
        # Add the number of the hex we clicked in white text
        cv2.putText(frame, str(hex_number), (x + 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.imshow('Hex maze video', frame)

def undo_last_click():
    """ Undo the last click by removing the coordinates from our list and the plot """
    global frame, clicked_points
    if clicked_points:  # Check if there are any clicked points to undo
        removed_hex_index = len(clicked_points) - 1
        removed_hex = hex_list[removed_hex_index]
        click_to_undo = clicked_points.pop()  # Remove the last clicked point
        print(f"Removed hex {removed_hex} at ({click_to_undo[0]}, {click_to_undo[1]})")
        # Redraw the frame (so we can remove the dot and hex label for the click we undid)
        frame[:] = original_frame.copy()
        # Redraw all remaining dots and hex labels
        for i, point in enumerate(clicked_points):
            cv2.circle(frame, point, radius=5, color=(0, 0, 255), thickness=-1)
            cv2.putText(frame, str(hex_list[i]), (point[0] + 10, point[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.imshow('Hex maze video', frame)

# Load the video file
cap = cv2.VideoCapture(video_file_path)

# Complain if we can't find or open the video file
if not cap.isOpened():
    print(f"Error: Could not open video at {video_file_path}!")
    exit()

# We only need a single frame from the video to get hex positions
# Feel free to change frame_number and try again if this frame is bad (person blocking hex view, etc)
frame_number = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)  # Set the frame position
ret, frame = cap.read()

# Complain if we can't read this frame
if not ret:
    print(f"Error: Could not read frame {frame_number}!")
    cap.release()
    exit()

# Set up the window and attach the mouse callback function
cv2.namedWindow('Hex maze video')
cv2.setMouseCallback('Hex maze video', get_pixel_coordinates)

# Display the selected frame
cv2.imshow('Hex maze video', frame)
original_frame = frame.copy() # Save a copy to use for redrawing if we undo clicks

# Run until the user closes the window
while True:
    key = cv2.waitKey(1) & 0xFF
    # Press 'q' or 'escape' to close the window
    if key == ord('q') or key == 27:
        break
     # Press 'u' to undo the last click
    elif key == ord('u'): 
        undo_last_click()

# Close all windows 
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)  # Add a short delay so the window actually closes

### Save clicked output to a csv file

This is in a separate code block so we don't save prematurely.

Set `save = True` to actually save.
Saving will overwrite existing files with the same name, so `save = False` by default as a safeguard.

It's probably a good idea to set a descriptive `session_name` to avoid this.

In [ ]:
# If we want to save the file (set this to True)
save = False

# File will be saved as {session_name}_hex_coordinates.csv
session_name = ""

# Check that we have clicked to indicate exactly one centroid per hex
if len(clicked_points) != len(hex_list):
    print(f"Expected centroids for {len(hex_list)} hexes, but {len(clicked_points)} hexes were clicked!")
    print(f"Please go back and click centroids for exactly {len(hex_list)} hexes.")

# Create a dataframe to save the x, y coordinates for each hex
x_coords, y_coords = zip(*clicked_points)
hex_coordinates_df = pd.DataFrame({'hex': hex_list, 'x': x_coords, 'y': y_coords})

# Optionally, convert the x, y coords in pixels to meters and add that to the dataframe
if CM_PER_PIXEL is not None:
    meters_per_pixel = CM_PER_PIXEL / 100
    hex_coordinates_df['x_meters'] = np.array(x_coords)*CM_PER_PIXEL
    hex_coordinates_df['y_meters'] = np.array(y_coords)*CM_PER_PIXEL
display(hex_coordinates_df)

# Save the dataframe to a csv
if save:
    save_file_name = f'{session_name}_hex_coordinates'
    hex_coordinates_df.to_csv(f"{save_file_name}.csv", index=False)
    print(f"Hex coordinates saved to {save_file_name}")